In [2]:
# Import required packages
import pandas as pd
import numpy as np
import os
import docplex.mp
from docplex.mp.model import Model

# Setting up optimisation engine for solving on local machine
# Add path to CPLEX solver engine
import sys
cplex_path = "C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio_Community1210\\cplex\\python\\3.7\\x64_win64"
sys.path.insert(0, cplex_path)
import cplex

In [21]:
# Obtain elements data from csv file previously downloaded into Pandas dataframe
while os.getcwd().endswith("mailc") == False:
    os.chdir("..")
os.chdir(os.getcwd() + "/R/FPL Player Season History")
tbl = pd.read_csv("test_output.csv")
elements = pd.read_csv("Elements.csv")
teams_pd = pd.read_csv("Teams.csv")

In [4]:
# One hot encode player type and team
positions = tbl['element_type'].to_numpy()
teams = tbl['team_code'].to_numpy()
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder = OneHotEncoder(sparse = False)

# Reshape numpy arrays to rows by column
positions = positions.reshape(len(positions), 1)
teams = teams.reshape(len(teams), 1)

# Binary matrices for positions and teams
binaryPositions = oneHotEncoder.fit_transform(positions)
binaryTeams = oneHotEncoder.fit_transform(teams)

In [5]:
# Create model object
m = Model(name = "Fantasy Premier League Team Selection")

# Decision variables: in a binary list, each item represents whether the player linked to index is selected in FPL team
indexList = tbl.index.tolist()
pick = m.binary_var_list(indexList)

# Add constraints
# - Team must cost less than 100m (==1000 units)
budget = 1000
m.add_constraint(m.sum(pick[i]*tbl.at[i, 'now_cost'] for i in indexList) <= budget)
# - Team must have exactly 15 players
players = 15
m.add_constraint(m.sum(pick[i] for i in indexList) == players)
# - Each position must have fixed number of players
posLimit = [2, 5, 5, 3]
for k in range(0, 4):
    m.add_constraint(m.sum(pick[i]*binaryPositions[i, k] for i in indexList) == posLimit[k])
# - Total number of players picked from each team is capped at 3
teamLimit = 3
for k in range(0, 20):
    m.add_constraint(m.sum(pick[i]*binaryTeams[i, k] for i in indexList) <= teamLimit)


# Define objective function
m.maximize(m.sum(pick[i]*tbl.at[i, 'points_per_game'] for i in indexList))

In [6]:
m.print_information()

Model: Fantasy Premier League Team Selection
 - number of variables: 628
   - binary=628, integer=0, continuous=0
 - number of constraints: 26
   - linear=26
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [25]:
# Solve model - solution contains table indices. Indices are passed to original table to find player IDs
sol = m.solve(url = None, key = None)
s = sol.as_df()
solIndex = s['name'].to_numpy().astype('int64')
team = pd.DataFrame(columns = ['id','player_name', 'position','expected_points','team', 'cost'])
positions = ['GK', 'DF', 'MF', 'ST']
for i in range(0, len(solIndex)):
    pid = tbl.at[solIndex[i], 'id']
    name = elements[elements['id'] == pid]['web_name'].values[0]
    position = positions[elements[elements['id'] == pid]['element_type'].values[0].astype('int64')-1]
    points = tbl.at[solIndex[i], 'points_per_game']
    t = teams_pd[teams_pd['code'] == tbl.at[solIndex[i], 'team_code']]['name'].values[0]
    c = tbl.at[solIndex[i], 'now_cost']/10
    team = team.append({'id':pid, 'player_name':name, 'position':position, 'expected_points':points, 'team':t, 'cost': c}, ignore_index = True)
print('Total Points: ' + team['expected_points'].sum().astype('str'))
print('Total Cost: ' + team['cost'].sum().astype('str'))
team

Total Points: 84.29999999999998
Total Cost: 100.0


,id,player_name,position,expected_points,team,cost
0,617,Marí,DF,7.0,Arsenal,4.8
1,93,Pope,GK,4.4,Burnley,4.9
2,103,Alonso,DF,7.0,Chelsea,6.2
3,166,Vardy,ST,6.4,Leicester,9.7
4,215,De Bruyne,MF,6.8,Man City,10.6
5,233,Rashford,ST,6.1,Man Utd,8.8
6,239,Martial,MF,5.3,Man Utd,8.0
7,618,Fernandes,MF,7.6,Man Utd,8.6
8,269,Shelvey,MF,4.1,Newcastle,4.9
9,281,McGovern,GK,5.5,Norwich,4.1
